In [1]:
import time
import gc
import warnings
import random
import torch
import numpy as np
import networkx as nx
import scipy.sparse as sparse
from utils import *
from model import *
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import normalize
warnings.filterwarnings("ignore")

print(torch.__version__)
print(torch.cuda.is_available())

1.7.0
True


In [2]:
def pre_sample_for_ether(walk_times,adj_sparse, train_index, batch_size, save_name, do_walk):
    if do_walk:
        nodes_num = len(train_index)
        walks = torch.zeros(nodes_num, walk_times+1).long().cuda()
        batches = create_batches_forList(train_index, batch_size, True)
        i=0
        degrees = torch.tensor(adj_sparse.sum(1)).view(-1).cuda()
        candi_node = 0
        for batch in batches:
            walks[i*batch_size : i*batch_size + len(batch), 0] = torch.tensor(batch).cuda()
            candi_node = sparse_mx_to_torch_sparse_tensor(adj_sparse[batch.cpu().numpy()]).cuda()
            chosen_node = torch.zeros(len(batch), adj_sparse.shape[0]).cuda()
            for id in range(len(batch)):
                chosen_node[id][batch[id]] = 1.
            candi_node = ((- chosen_node + candi_node)> 0.0).float()
            for walk_id in range(walk_times):
                for x in range(candi_node.shape[0]):
                    if candi_node[x].sum()==0:
                        candi_node[x][batch[x]] = 1.
                p = candi_node * degrees
                print("p = {}".format(p))
                new_node = torch.multinomial(p,1).squeeze(1)
                walks[i*batch_size : i*batch_size + len(batch), walk_id+1] = new_node
                for id in range(len(batch)):
                    chosen_node[id][new_node[id]] = 1.
                candi_node = candi_node - chosen_node + sparse_mx_to_torch_sparse_tensor(adj_sparse[new_node.cpu()]).cuda()
                candi_node = (candi_node> 0.0).float()
                
            i+=1
        torch.cuda.empty_cache()
        result1 = np.array(walks.cpu())
        io.savemat(save_name+'.mat',{save_name:result1})
        return walks
    else:
        print("load path: {}".format(save_name + '.mat'))
        walks = io.loadmat(save_name+'.mat')
        return torch.tensor(walks[save_name]).cuda()

In [3]:
class Train_Model_Ether():
    def __init__(self, args):
        self.args = args
        self.model = AttnWeight(self.args).cuda()
        self.sparse_adj, self.sparse_adj_train,self.sparse_adj_train_all, self.features, self.train_feature, self.train_feature_all,self.labels, self.train_labels, self.id_train, self.id_valid, self.id_test, num_labels = Origin_load_ether_data()
        self.eval_labels = self.labels[self.id_valid]
        self.test_labels = self.labels[self.id_test]
        self.sample_batch_size = 256
        print('train walk')
        name = 'seed='+str(self.args.seed)+'_walk='+str(self.args.time)
#         self.walks_train = pre_sample_for_ether(self.args.time, self.sparse_adj_train, self.id_train, self.sample_batch_size, args.walk_path + 'train_'+name,args.do_walk)
#         print('valid walk')
#         self.walks_valid = pre_sample_for_ether(self.args.time, self.sparse_adj, self.id_valid, self.sample_batch_size,args.walk_path + 'valid_'+name,args.do_walk)
        self.sparse_adj.setdiag(1.0) 
        self.sparse_adj_train.setdiag(1.0) 
        self.nor_graph = normalize(self.sparse_adj, norm='l1', axis=1)
        self.nor_graph_train = normalize(self.sparse_adj_train, norm='l1', axis=1)
        self.nor_graph_train_all = normalize(self.sparse_adj_train_all, norm='l1', axis=1)
        self.args.feature_dim = self.features.shape[1]
        self.args.num_nodes = num_labels
        self.args.num_labels = num_labels
        print('cuda ready')
        self.logs = create_logs(self.args)
        self.best_loss = 1000
        self.best_acc = 0
        self.best_loss_both = 1000
        self.best_acc_both = 0
        self.stop_count = 0
        self.best_loss_epoch = -1
        self.best_both_epoch = -1
        self.best_acc_epoch  = -1
        self.epoch_idx = 0
        self.total_time = 0.0

    def fit(self):
        print("\nTraining started.\n")
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.args.learning_rate, weight_decay = self.args.weight_decay)
        self.optimizer.zero_grad()
#         batches = create_batches_forWalk(self.walks_train, self.args.batch_size)
#         valid_batch = create_batches_forWalk(self.walks_valid, self.args.batch_size)
        self.id_train = list(self.id_train)
        for epoch in tqdm(range(self.args.epochs)):
            self.model.train()
            # train
            predictions = self.model(self.nor_graph_train_all, self.train_feature_all, self.args.time, self.sparse_adj_train, self.id_train, self.sample_batch_size)
            prediction_loss = calculate_predictive_loss(self.train_labels, predictions)
            acc, label_pre = calculate_reward(self.train_labels, predictions)
            self.optimizer.zero_grad()
            prediction_loss.backward()
            self.optimizer.step()
            # eval
            self.model.eval()
            eval_predictions = self.model(self.nor_graph, self.features, self.args.time, self.sparse_adj, self.id_valid, self.sample_batch_size)
            eval_prediction_loss = calculate_predictive_loss(self.eval_labels, eval_predictions)
            eval_acc, eval_label_pre = calculate_reward(self.eval_labels, eval_predictions)
            print("Train: loss = {} acc = {} | Valid: loss = {} acc = {}".format(prediction_loss, acc, eval_prediction_loss, eval_acc))
            
#             for batch in valid_batch:
#                 label = torch.index_select(self.labels, 0, batch[:,0].view(-1))
#                 loss_node, acc, label_predict = self.process_node(label, batch, self.features, self.nor_graph)
# #                 print("loss_node",loss_node,"acc:",acc,"label_predict:",label_predict)
#                 valid_loss += loss_node.item()
#                 valid_acc += acc.item()
                
#             valid_loss = round(valid_loss*1000/len(self.id_valid), 4)
#             valid_acc = round(valid_acc/len(self.id_valid), 4)
#             self.acc_score = round(self.acc_score/len(self.id_train), 4)
#             loss_score = round(self.epoch_loss*1000/len(self.id_train), 4)
#             if epoch % 1 == 0:                 
#                 print("epoch",epoch,"loss_train:",prediction_loss,"acc_train:",
#                                                      acc,'||',"loss_valid:",eval_prediction_loss,"acc_valid:",eval_acc)
            if  eval_prediction_loss  < self.best_loss :
                self.best_loss = eval_prediction_loss
                torch.save(self.model.state_dict(),self.args.save_path_loss+"best_model.pt")
                stop_count = 0
                self.best_loss_epoch = epoch
            else:
                stop_count += 1
                if stop_count == self.args.patience:
                    print(self.args.patience, "times no decrease")
                    return round(0/(epoch+1), 4), round(0 / (epoch+1), 4), time.time()
        print('Max epoches reaches!')
        return round(0/(epoch+1), 4), round(0 / (epoch+1), 4), time.time()
            
    def evaluation(self):
        loss_result = torch.zeros(len(self.id_test), self.args.eva_times, dtype=torch.long)
        acc_result = torch.zeros(len(self.id_test), self.args.eva_times, dtype=torch.long)
        loss_acc = 0.0
        acc_acc = 0.0
        #print('test walk')
        name = 'seed='+str(self.args.seed)+'_walk='+str(self.args.time)
#         self.walks_test = pre_sample_for_ether(self.args.time, self.sparse_adj, self.id_test, self.sample_batch_size,self.args.walk_path + 'test_'+name,False)
#         test_batch = create_batches_forWalk(self.walks_test, self.args.batch_size)

        self.model.eval()
        self.model.load_state_dict(torch.load(self.args.save_path_loss+"best_model.pt"))
#         for batch in test_batch:
#             label = torch.index_select(self.labels, 0, batch[:,0].view(-1))
#             loss_node, acc, label_predict = self.process_node(label, batch, self.features, self.nor_graph)
#             loss_acc += loss_node.item()
#             acc_acc += acc.item()

#         loss_acc = round(loss_acc*1000/len(self.id_test), 4)
#         acc_acc = round(acc_acc/len(self.id_test), 4)
        test_predictions = self.model(self.nor_graph, self.features, self.args.time, self.sparse_adj, self.id_test, self.sample_batch_size)
        test_prediction_loss = calculate_predictive_loss(self.test_labels, test_predictions)
        test_acc, test_label_pre = calculate_reward(self.test_labels, test_predictions)
        test_prediction_loss, test_acc = test_prediction_loss.detach().cpu().numpy(), test_acc.detach().cpu().numpy()
        print("loss acc:", test_prediction_loss,"acc_acc",test_acc, 'load epoch:', self.best_loss_epoch)
        return  test_prediction_loss, test_acc

    def process_node(self,label, node, feature, graph):
        prediction = self.model(node, graph, feature)
        prediction_loss = calculate_predictive_loss(label, prediction)
        acc, label_pre = calculate_reward(label, prediction)
        return prediction_loss, acc, label_pre

    def process_batch(self,label,batch, feature, graph):
        self.optimizer.zero_grad()
        pre = torch.cuda.memory_allocated()
        batch_loss, acc, label_pre = self.process_node(label, batch, feature, graph)
        self.acc_score += acc.item()
        batch_loss.backward()
        self.optimizer.step()
        return batch_loss.item()

In [4]:
class argument():
    def __init__(self):
        self.dataset = 'ether'
        self.gamma = 1
        self.hidden = 128
        self.time = 3
        self.save_path_acc = "./best_ether_attn_feat_matrix/acc/"
        self.save_path_loss = "./best_ether_attn_feat_matrix/loss/"
        self.save_path_both = "./best_ether_attn_feat_matrix/both/"
        self.walk_path = "./walks_ether_attention_feat_matrix/"
        self.patience = 30
        self.batch_size = 256
        self.epochs = 1000
        self.learning_rate = 0.001
        self.weight_decay = 0
        self.num_labels = 41
        self.feature_dim = 12
        self.walk_times = 10
        self.eva_times = 1
        self.seed = 2019
        self.dropout = 0
        self.way = 'both'
        self.conbime = 'Lstm'
        self.do_walk = False
args = argument()

In [ ]:
repeat_time = 10
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
ways = ['width']
loss_loses = []
loss_accs = []
conbimes = ['Mean']
print('Different walk time')
#walk_timess = [0,4,5,6,7,8,9]
for walk_time in tqdm(range(4,10)):
    loss_accs = []
    acc_accs = []
    epoch_times = []
    batch_times = []
    total_times = []
    print('Different seed')
    for seed in tqdm(range(10,20)):
        args = argument()
        args.seed = seed
        args.time = walk_time
        torch.manual_seed(args.seed)
        random.seed(args.seed)
        torch.cuda.manual_seed(args.seed)
        torch.backends.cudnn.deterministic = True
        np.random.seed(args.seed)
        args.save_path_acc = "./best_ether_attn_feat_matrix/acc/"
        args.save_path_loss = "./best_ether_attn_feat_matrix/loss/"
        args.save_path_acc = args.save_path_acc+ 'ether' + '_dropout='+str(args.dropout)+'_wd='+str(args.weight_decay)+ '_b='+str(args.batch_size)+  '_lr='+str(args.learning_rate)+ '_epochs='+str(args.epochs)+'_walk='+ str(args.time)+'_'
        args.save_path_loss = args.save_path_loss+ 'ether' + '_dropout='+str(args.dropout)+ '_wd='+str(args.weight_decay)+ '_b='+str(args.batch_size)+  '_lr='+str(args.learning_rate)+'_epochs='+str(args.epochs)+ '_walk='+ str(args.time)+'_'
        model = Train_Model_Ether(args)
        epoch_time, batch_time, total_time = model.fit()
        loss_acc,acc_acc = model.evaluation()
        epoch_times.append(epoch_time)
        total_times.append(total_time)
        batch_times.append(batch_time)
        loss_accs.append(loss_acc)
        acc_accs.append(acc_acc)
        del model
        gc.collect()
        torch.cuda.empty_cache()
    print('#####################################')
    print("walk_time:", args.time, 'ways:', args.way, 'conbime：', args.conbime)
    print('batch_times:',sum(batch_times)/repeat_time)
    print('epoch_times:',sum(epoch_times)/repeat_time)
    print('total_times:',sum(total_times)/repeat_time)
    print('loss_accs:',sum(loss_accs)/repeat_time,loss_accs)
    print('acc_accs:',sum(acc_accs)/repeat_time,acc_accs)
    print(sum(batch_times)/repeat_time, sum(epoch_times)/repeat_time, sum(loss_accs)/repeat_time, sum(acc_accs)/repeat_time)
    print('#####################################')

Different walk time


Different seed


features = (1402220, 12) type = <class 'numpy.ndarray'>
train_index 572
val_index 82
test_index 162
adj torch.Size([1402220, 1402220])
features torch.Size([1402220, 12])
train walk
cuda ready

Training started.



Train: loss = 7.715138912200928 acc = 0.008741258643567562 | Valid: loss = 7.637262344360352 acc = 0.03658536449074745
Train: loss = 6.539260387420654 acc = 0.02622377499938011 | Valid: loss = 7.2444844245910645 acc = 0.060975607484579086
Train: loss = 5.865132808685303 acc = 0.038461536169052124 | Valid: loss = 6.314367294311523 acc = 0.060975607484579086
Train: loss = 5.189637184143066 acc = 0.050699301064014435 | Valid: loss = 5.968329906463623 acc = 0.0731707289814949
Train: loss = 4.5876617431640625 acc = 0.07342657446861267 | Valid: loss = 5.502170562744141 acc = 0.0731707289814949
Train: loss = 4.050677299499512 acc = 0.10839160531759262 | Valid: loss = 5.380706787109375 acc = 0.09756097197532654
Train: loss = 3.7269556522369385 acc = 0.14335663616657257 | Valid: loss = 5.219001770019531 acc = 0.09756097197532654
Train: loss = 3.434462785720825 acc = 0.21503496170043945 | Valid: loss = 4.581601142883301 acc = 0.08536584675312042
Train: loss = 3.015364646911621 acc = 0.2692307531